<a href="https://colab.research.google.com/github/MaruthiMax/INFO-5731_SPRING2024/blob/main/lingampalli_maruthi_Exercis_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 In-class Exercise 4**

**This exercise will provide a valuable learning experience in working with text data and extracting features using various topic modeling algorithms. Key concepts such as Latent Dirichlet Allocation (LDA), Latent Semantic Analysis (LSA), lda2vec, and BERTopic.**

***Please use the text corpus you collected in your last in-class-exercise for this exercise. Perform the following tasks***.

**Expectations**:
*   Students are expected to complete the exercise during lecture period to meet the active participation criteria of the course.
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

**Total points**: 40

**Deadline**: This in-class exercise is due at the end of the day tomorrow, at 11:59 PM.

**Late submissions will have a penalty of 10% of the marks for each day of late submission, and no requests will be answered. Manage your time accordingly.**


## Question 1 (10 Points)

**Generate K topics by using LDA, the number of topics K should be decided by the coherence score, then summarize what are the topics.**

You may refer the code here: https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [1]:
# Write your code here
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [2]:
!pip install pyLDAvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 17.7 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.2.1 which is incompatible.


In [57]:
# import necessary liraries
import re
import numpy as np
import pandas as pd
from pprint import pprint


import gensim
from gensim import corpora
from gensim.models import CoherenceModel
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()


import re
import numpy as np
import string

#downloading pacages punkt and stopwords



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [58]:

# taking a article text for the model
article = """
In recent years, the fashion industry has faced growing scrutiny over its environmental impact, from excessive water usage to the disposal of unsold clothing in landfills. Enter GreenThreads, a visionary company poised to tackle these challenges head-on. Founded by a group of environmental enthusiasts and fashion designers, GreenThreads has embarked on a mission to redefine sustainable fashion.

At the core of GreenThreads' business model is its commitment to using only eco-friendly materials. Organic cotton, known for its minimal environmental footprint, is the primary fabric in their collections. By avoiding the pesticides and synthetic fertilizers used in conventional cotton farming, GreenThreads significantly reduces water pollution and health hazards to workers. Furthermore, the company has innovated a line of clothing made from bamboo fibers, which boast natural antibacterial properties and require less water to grow compared to other crops.

But GreenThreads doesn't stop there. Recognizing the problem of plastic waste, the company has developed a process for creating recycled polyester from plastic bottles, giving new life to what would otherwise be pollution. This initiative not only cleans up the environment but also provides customers with durable and water-resistant clothing options.

Transparency is another cornerstone of GreenThreads' philosophy. The company meticulously tracks and shares its supply chain details, from the sourcing of raw materials to the ethical treatment of workers in its factories. This level of openness has cultivated a loyal customer base that values the integrity behind each purchase.
"""


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [59]:


# function for preprocessing given text
def preprocess_of_the_text(article):
    # Removed  any punctuation marks and making all the text to lower case
    article = article.translate(str.maketrans('', '', string.punctuation)).lower()
    #making small words(tokens)
    tokens_a = word_tokenize(article)
    #removing stop words from the
    stop_words_a = set(stopwords.words('english'))
    tokens_a = [b for b in tokens_a if b not in stop_words_a]
    return tokens_a


# Tokenize and preprocess the text
tokens_of_text = preprocess_of_the_text(article)
dict_of_text = corpora.Dictionary([tokens_of_text])
corp_of_text = [dict_of_text.doc2bow(tokens_of_text)]

# defining a function for calucuate coherence scores for eaxh topic of the article given as input
def coher_scores_of_text(corp_of_text, dict_of_text, article_text, limit, start=2, step=1):
    coher_scores = []
    for no_of_topics in range(start, limit, step):
        text_model = gensim.models.LdaModel(corpus=corp_of_text, id2word=dict_of_text, num_topics=no_of_topics)
        coher_of_emodel = CoherenceModel(model=text_model, texts=article_text, dictionary=dict_of_text, coherence='c_v')
        textmodel_coherence_score = coher_of_emodel.get_coherence()
        coher_scores.append((no_of_topics, textmodel_coherence_score))
        print(f'The coherence score for {no_of_topics} topics: {textmodel_coherence_score:.4f}')
    return coher_scores


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [60]:
# here we are defining the main function of the model
def main():
    article_limit = 15
    global lda_model
    coher_scores = coher_scores_of_text(corp_of_text, dict_of_text, [tokens_of_text], limit=article_limit, start=2, step=1)
    k_no_of_topics, _= max(coher_scores, key=lambda x: x[1])
    lda_model = gensim.models.LdaModel(corpus=corp_of_text, id2word=dict_of_text, num_topics=k_no_of_topics)
    topic_article = lda_model.show_topics(formatted=False)

    print(f' number of topics k is : {k_no_of_topics}')

    for id, topic in topic_article:
        words_text = [words_text for words_text, weight in topic]
        top_words = ", ".join([f"{words_text} ({weight:.4f})" for words_text, weight in topic])
        print(f'the topic {id + 1}: {top_words}')


# here we are calling the main function
if __name__ == "__main__":
    main()

# we are visulaizing the topics and their coherence importance in topic selction
visualise_data = pyLDAvis.gensim.prepare(lda_model, corp_of_text, dict_of_text)
pyLDAvis.display(visualise_data)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


The coherence score for 2 topics: 0.6088
The coherence score for 3 topics: 0.5519


The coherence score for 4 topics: 0.5548


The coherence score for 5 topics: 0.5480


The coherence score for 6 topics: 0.5655


...

The coherence score for 10 topics: 0.5369


The coherence score for 11 topics: 0.5555


The coherence score for 12 topics: 0.5526


The coherence score for 13 topics: 0.5430


The coherence score for 14 topics: 0.5018
 number of topics k is : 2
the topic 1: greenthreads (0.0249), company (0.0175), water (0.0172), fashion (0.0165), clothing (0.0154), environmental (0.0150), cotton (0.0122), workers (0.0119), pollution (0.0114), plastic (0.0103)
the topic 2: greenthreads (0.0264), company (0.0191), environmental (0.0143), clothing (0.0139), fashion (0.0128), materials (0.0123), water (0.0121), plastic (0.0117), pollution (0.0105), workers (0.0100)


## Question 2 (10 Points)

**Generate K topics by using LSA, the number of topics K should be decided by the coherence score, then summarize what are the topics.**

You may refer the code here: https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

In [32]:

# Import necessary libraries
import pandas as pd
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation, preprocess_string, strip_short, stem_text
from gensim import corpora
from gensim.models import LsiModel
from gensim.models.coherencemodel import CoherenceModel

# Sample data
sample_text = [
    "The briyani in irving tastes so good, but the prices are a bit high compared to Inida",
    "Envinorment is becoming worse day by day due to pollution caused by planes and automobiles.",
    " India soon to launch UPI method pay method across the worls with its digital currrency policy",
]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [33]:
# Creat a DataFrame for the sample text
df = pd.DataFrame({'Review': sample_text})

# Preprocess the text using the prepocess the text function
def preprocess_the_text(text):
    cus_filters = [lambda x: x.lower(), remove_stopwords, strip_punctuation, strip_short, stem_text]
    text = preprocess_string(text, cus_filters)
    return text

# Apply the preprocessing to the sample text
df['Review_Text (Clean)'] = df['Review'].apply(lambda x: preprocess_the_text(x))

# Create a dictionary with the corpus
corpus_text = df['Review_Text (Clean)']
dictionary_text = corpora.Dictionary(corpus_text)

# Convert corpus into a bag of words
bag_of_words = [dictionary_text.doc2bow(text) for text in corpus_text]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [36]:
#finding the coherencve scores for each text in the given sentences
for i in range(2, 11):
    lsi = LsiModel(bag_of_words, num_topics=i, id2word=dictionary_text)
    coherence_of_model = CoherenceModel(model=lsi, texts=df['Review_Text (Clean)'], dictionary=dictionary_text, coherence='c_v')
    coherence_scores = coherence_of_model.get_coherence()
    print('Coherence score with {} clusters: {}'.format(i, coherence_scores))

# Perform SVD on the bag of words with the LsiModel to extract 2 topics
lsi = LsiModel(bag_of_words, num_topics=2, id2word=dictionary_text)

# Find the 5 words with the strongest association to the derived topics
for topics, words_in_topic in lsi.print_topics(num_words=10):
    print('Words in {}: {}.'.format(topics, words_in_topic))

# scores for the text
corpus_text


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gensim/models/lsimodel.py:963: DeprecationWarning: Please use `csc_matvecs` from the `scipy.sparse` namespace, the `scipy.sparse.sparsetools` namespace is deprecated.
  sparsetools.csc_matvecs(
/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning: 'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.
  out = (1 - tri(m.shape[0], m.shape[1], k - 1, m.dtype.char)) * m
/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning: 'tri'/'tril/'triu' 

Coherence score with 2 clusters: 0.1975266915775667
Coherence score with 3 clusters: 0.26384705173352


/usr/local/lib/python3.10/dist-packages/gensim/models/lsimodel.py:963: DeprecationWarning: Please use `csc_matvecs` from the `scipy.sparse` namespace, the `scipy.sparse.sparsetools` namespace is deprecated.
  sparsetools.csc_matvecs(
/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning: 'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.
  out = (1 - tri(m.shape[0], m.shape[1], k - 1, m.dtype.char)) * m
/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning: 'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.
  out = (1 - tri(m.shape[0], m.shape[1], k - 1, m.dtype.char)) * m
/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning: 'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13

Coherence score with 4 clusters: 0.3041857672520198
Coherence score with 5 clusters: 0.29716307953015014


/usr/local/lib/python3.10/dist-packages/gensim/models/lsimodel.py:963: DeprecationWarning: Please use `csc_matvecs` from the `scipy.sparse` namespace, the `scipy.sparse.sparsetools` namespace is deprecated.
  sparsetools.csc_matvecs(
/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning: 'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.
  out = (1 - tri(m.shape[0], m.shape[1], k - 1, m.dtype.char)) * m
/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning: 'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.
  out = (1 - tri(m.shape[0], m.shape[1], k - 1, m.dtype.char)) * m
/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning: 'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13

Coherence score with 6 clusters: 0.2971630795301503
Coherence score with 7 clusters: 0.29716307953015025
Coherence score with 8 clusters: 0.26675594578448


/usr/local/lib/python3.10/dist-packages/gensim/models/lsimodel.py:963: DeprecationWarning: Please use `csc_matvecs` from the `scipy.sparse` namespace, the `scipy.sparse.sparsetools` namespace is deprecated.
  sparsetools.csc_matvecs(
/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning: 'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.
  out = (1 - tri(m.shape[0], m.shape[1], k - 1, m.dtype.char)) * m
/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning: 'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.
  out = (1 - tri(m.shape[0], m.shape[1], k - 1, m.dtype.char)) * m
/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning: 'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13

Coherence score with 9 clusters: 0.2637444885168703
Coherence score with 10 clusters: 0.23063559004721845
Words in 0: 0.555*"method" + 0.277*"pai" + 0.277*"digit" + 0.277*"india" + 0.277*"launch" + 0.277*"currrenc" + 0.277*"upi" + 0.277*"soon" + 0.277*"worl" + 0.277*"polici".
Words in 1: -0.632*"dai" + -0.316*"automobil" + -0.316*"wors" + -0.316*"pollut" + -0.316*"plane" + -0.316*"caus" + -0.316*"envinor" + -0.000*"compar" + 0.000*"bit" + 0.000*"briyani".


/usr/local/lib/python3.10/dist-packages/gensim/models/lsimodel.py:963: DeprecationWarning: Please use `csc_matvecs` from the `scipy.sparse` namespace, the `scipy.sparse.sparsetools` namespace is deprecated.
  sparsetools.csc_matvecs(
/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning: 'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.
  out = (1 - tri(m.shape[0], m.shape[1], k - 1, m.dtype.char)) * m
/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning: 'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.
  out = (1 - tri(m.shape[0], m.shape[1], k - 1, m.dtype.char)) * m
/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning: 'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13

0    [briyani, irv, tast, good, price, bit, high, c...
1    [envinor, wors, dai, dai, pollut, caus, plane,...
2    [india, soon, launch, upi, method, pai, method...
Name: Review_Text (Clean), dtype: object

## Question 3 (10 points):
**Generate K topics by using lda2vec, the number of topics K should be decided by the coherence score, then summarize what are the topics.**

You may refer the code here: https://nbviewer.org/github/cemoody/lda2vec/blob/master/examples/twenty_newsgroups/lda2vec/lda2vec.ipynb

In [37]:
# importing necessary libraies
import pyLDAvis
pyLDAvis.enable_notebook()
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
#initializing count vectorizer
vector_text = CountVectorizer()

# we are using the article as sample text used in previously
a= vector_text.fit_transform(sample_text)

# Get the top words for each topic
count_words = 10
topwords_in_topic= {}
for b in range(b.shape[0]):
  word_index = np.argsort(x[b].toarray()[0])[::-1][:count_words]
  topword = [vector_text.get_feature_names_out()[k] for k in word_index]
  out = 'Topic %i has top words: %s' % (k, ', '.join(topword))
  print(out)
  topwords_in_topic[k] = topword

Topic 0 has top words: the, briyani, prices, irving, inida, but, bit, in, so, high
Topic 1 has top words: by, day, pollution, envinorment, due, is, caused, planes, worse, becoming
Topic 2 has top words: method, india, policy, currrency, digital, worls, its, launch, pay, across


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Question 4 (10 points):
**Generate K topics by using BERTopic, the number of topics K should be decided by the coherence score, then summarize what are the topics.**

You may refer the code here: https://colab.research.google.com/drive/1FieRA9fLdkQEGDIMYl0I3MCjSUKVF8C-?usp=sharing

In [39]:
!pip install bertopic

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 27.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl.metadata (3.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 10.6 MB/s eta 0:00:00
Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━

In [54]:
text_rand = [
    "Transportation sector known significant contribution pollution seen surge adoption sustainable mobility solution electric bus car sharing micro mobility emerged key player reshaping navigate urban landscape.",
    "Car sharing program enable individual access vehicle needed basis eliminating need personal car ownership.",
    "Benefit car sharing extend beyond environmental conservation include reduced traffic congestion minimized parking space requirement lower overall transportation cost user.",
    "Continued innovation investment advocacy pave way future mobility efficient accessible also sustainable environmentally conscious.",
    "Furthermore incorporation electric hybrid vehicle within car sharing fleet bolstered sustainability quotient offering user eco friendly alternative transportation need.",
    "Fostering community empowerment education engagement form cornerstone successful sustainable mobility solution.",
    "Challenge opportunity horizon.",
    "Investment research development coupled strategic partnership government business community pave way seamless integration sustainable mobility solution fabric urban life.",
    "Embracing capability electric bus car sharing micro mobility enable city cultivate transportation ecosystem cleaner efficient also inclusive benefiting environment people inhabit.",
    "Moreover eco friendly nature resonates increasing focus curbing carbon emission fostering sustainable development urban area.",
    "Educational program awareness campaign instrumental cultivating culture sustainability encouraging individual make informed choice transportation habit.",
    "Despite evident benefit sustainable mobility solution several challenge persist widespread implementation.",
    "Designed short distance travel alternative provide convenient eco conscious solution navigating congested urban area.",
    "Bustling city hum traditional bus gradually replaced whisper electric bus vehicle represent fundamental shift public transportation offering environmentally friendly alternative gas guzzling counterpart.",
    "Role infrastructure policy promoting sustainable mobility.",
    "Rising popularity micro mobility option attributed accessibility flexibility user friendly nature.",
    "Incorporating community input planning development sustainable mobility project city ensure effort effective also inclusive community centric.",
    "Proliferation solution positive step success contingent presence appropriate infrastructure well designed policy.",
    "Furthermore integration smart technology data driven solution hold potential optimize efficiency public transportation network enhance user experience car sharing service streamline operation micro mobility initiative.",
    "Furthermore policy initiative aimed incentivizing use sustainable mobility solution tax benefit electric vehicle user integration car sharing service urban planning strategy significantly influence consumer behavior encourage transition towards sustainable transportation practice."
]

# Now 'selected_sentences' is a Python list containing the 20 random sentences.


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [55]:
# importing the bert library
from bertopic import BERTopic


#crearing the bert model below
BERT_model = BERTopic(nr_topics=2, calculate_probabilities=True, verbose=True)


topic, _ = BERT_model.fit_transform(text_rand)

model_overview = BERT_model.get_topic_freq()

for n, b in model_overview[1:].values:
    bert_model_words = BERT_model.get_topic(n)
    bert_model_summary = ", ".join([word[0] for word in bert_model_words[:5]])
    print(f"Topic {n}: {bert_model_summary} (Freq: {b})")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
2024-03-30 03:48:09,349 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-03-30 03:48:11,451 - BERTopic - Embedding - Completed ✓
2024-03-30 03:48:11,456 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-03-30 03:48:14,457 - BERTopic - Dimensionality - Completed ✓
2024-03-30 03:48:14,466 - BERTopic - Cluster - Start clustering the reduced embeddings
/usr/local/lib/python3.10/dist-packages/hdbscan/hdbscan_.py:1170: DeprecationWarning: `alltrue` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `all` instead.
  self._all_finite = is_finite(X)
2024-03-30 03:48:14,474 - BERTopic - Cluster - Completed ✓
2024-03-30 03:48:14,478 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-03-30 03:48:14,501 - BERTopic - Representation - Completed ✓
2024-03-30 03:48:14,506 - BERTopic - Topic reduction - Reducing number of topics
2024-03-30 03:48:14,509 - BERTopic - Topic reduction - Reduced number of topics from 1 to 1


In [ ]:
# Write your code here


In [ ]:
# Write your code here


In [ ]:
# Write your code here


## Extra Question (5 Points)

**Compare the results generated by the four topic modeling algorithms, which one is better? You should explain the reasons in details.**

**This question will compensate for any points deducted in this exercise. Maximum marks for the exercise is 40 points.**

In [56]:
# Write your code here
"""
BERTopic makes recourse to BERT embeddings to identify refinements and deep semantic similarities in the written content that might pass unobserved by typical bag-of-words models. If dealing with intricate or complex information in texts, approach may end up in subjects deemed far more significant and cohesive.

In model analysing and In utilizing the vast knowledge of the pre-trained BERT framework, it can adjust to a broad range of domains and languages.

And also BERTopic, tends to yield better defined areas which can be more straightforward for individuals to recognize as topic compared to often more abstract topics generated via systems for instance LDA.

At the cutting edge of NLP, it's an updated approach for exposed modeling that can take advantages from ongoing innovations in language models.
"""

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


"\nBERTopic makes recourse to BERT embeddings to identify refinements and deep semantic similarities in the written content that might pass unobserved by typical bag-of-words models. If dealing with intricate or complex information in texts, approach may end up in subjects deemed far more significant and cohesive.\n\nIn model analysing and In utilizing the vast knowledge of the pre-trained BERT framework, it can adjust to a broad range of domains and languages.\n\nAnd also BERTopic, that utilizes HDBSCAN for clustering, tends to yield better defined areas which can be more straightforward for individuals to recognize as topic compared to often more abstract topics generated via systems for instance LDA.\n\nAt the cutting edge of NLP, it's an updated approach for exposed modeling that can take advantages from ongoing innovations in language models.\n"

# Mandatory Question

**Important: Reflective Feedback on this exercise**

Please provide your thoughts and feedback on the exercises you completed in this assignment.

Consider the following points in your response:

**Learning Experience:** Describe your overall learning experience in working with text data and extracting features using various topic modeling algorithms. Did you understand these algorithms and did the implementations helped in grasping the nuances of feature extraction from text data.

**Challenges Encountered:** Were there specific difficulties in completing this exercise?

Relevance to Your Field of Study: How does this exercise relate to the field of NLP?

**(Your submission will not be graded if this question is left unanswered)**



In [61]:
# Your answer here (no code for this question, write down your answer as detail as possible for the above questions):

'''
Please write you answer here:
My learning experience from this assignment is really vast and also I have learned many new things, but the most important thing in this assignment is very difficult to understand the story of what is going on like either many other codes, and online sources, and even I went through stock overflow so some some challenges I facedreally really hard for me to tackle and I feel that this assignment will help me in learning about birth model, very inept and yes, this exercise really helps and also relates to the field of NLP even for my project work where I use the boat model and it also




'''

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'\nPlease write you answer here:\nMy learning experience from this assignment is really vast and also I have learned many new things, but the most important thing in this assignment is very difficult to understand the story of what is going on like either many other codes, and online sources, and even I went through stock overflow so some some challenges I facedreally really hard for me to tackle and I feel that this assignment will help me in learning about birth model, very inept and yes, this exercise really helps and also relates to the field of NLP even for my project work where I use the boat model and it also\n\n\n\n\n'